### Neo4j drivers for python

https://neo4j.com/docs/api/python-driver/5.0/api.html#api-documentation

In [4]:
!pip install neo4j

In [6]:
!pip install graphdatascience

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 16.1 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 8.2.3
    Uninstalling tenacity-8.2.3:
      Successfully uninstalled tenacity-8.2.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
streamlit 1.37.1 requires tenacity<9,>=8.1.0, but you have tenacity 9.0.0 which is incompatible.


In [1]:
from neo4j import GraphDatabase

### Connecting to the database

In [4]:
uri = "neo4j://localhost:7687"
driver = GraphDatabase.driver(uri, auth=("neo4j", "password"))
session =  driver.session()

### Cypher operations

In [8]:
def show_result(query):
    result = session.run(query)
    print(result.data())

In [28]:
def import_patent_query(filename):
    query_text = """
                        LOAD CSV WITH HEADERS FROM 'file:///"""+filename+"""' AS row
                        WITH row,
                             trim(replace(replace(row.publication_doc_number, '[', ''), ']', '')) AS pub_doc_number,
                             trim(replace(replace(row.invention_title, '[', ''), ']', '')) AS title,
                             trim(replace(replace(row.applicant_organization, '[', ''), ']', '')) AS applicant,
                             trim(replace(replace(row.publication_kind, '[', ''), ']', '')) AS pub_kind,
                             COALESCE(trim(replace(replace(row.inventor_name, '[', ''), ']', '')), '') AS inventors,
                             COALESCE(trim(replace(replace(row.inventor_location, '[', ''), ']', '')), '') AS inventor_locs
                        
                        // Create Patent node
                        MERGE (p:Patent {id: trim(replace(replace(row.publication_doc_number, '[', ''), ']', ''))}) 
                        SET p.title = row.invention_title, p.date = row.publication_date
                        
                        // Create Organization and relationship
                        MERGE (a:Organization {name: applicant})
                        MERGE (a)-[:FILED]->(p)
                        
                        // Create PatentKind node and link it to the patent
                        MERGE (k:PatentKind {type: pub_kind})
                        MERGE (p)-[:HAS_KIND]->(k)
                        
                        WITH p, split(replace(replace(row.inventor, '[', ''), ']', ''), ',') AS inventors
                        UNWIND inventors AS inventorName
                        WITH p, trim(inventorName) AS inventorName
                        WHERE inventorName <> '' AND inventorName <> 'Jr.' AND inventorName <> 'III'
                        
                        MERGE (i:Inventor {name: trim(inventorName)})
                        MERGE (i)-[:INVENTED]->(p);
                        """
    return query_text

def import_sbir_query(filename):
    query_text = """
                    LOAD CSV WITH HEADERS FROM 'file:///"""+filename+"""' AS row
                    WITH row,
                         trim(replace(replace(row.`Company`, '[', ''), ']', '')) AS company,
                         trim(replace(replace(row.`Award Title`, '[', ''), ']', '')) AS award_title,
                         trim(replace(replace(row.`Agency Tracking Number`, '[', ''), ']', '')) AS tracking_number,
                         trim(replace(replace(row.`PI Name`, '[', ''), ']', '')) AS pi_name
                    
                    // Merge Company with existing Organization nodes
                    MERGE (org:Organization {name: company})
                    
                    // Merge PI Name with existing Inventor nodes
                    MERGE (pi:Inventor {name: pi_name})
                    
                    // Create Award node
                    MERGE (award:SBIR_Award {tracking_number: tracking_number})
                    SET award.title = award_title
                    
                    // Link Company to Award
                    MERGE (org)-[:RECEIVED_AWARD]->(award)
                    
                    // Link PI Name to Award
                    MERGE (pi)-[:PI_FOR]->(award);
                    """
    return query_text


In [12]:
wipe_query = "MATCH (n) DETACH DELETE n;"

remove_na_org = """
                MATCH (a:Organization {name: "N/A"})  
                DETACH DELETE a;
                """

## Patent data chunk upload, full dataset causes crashes

In [ ]:
# session.run(wipe_query)
# print('Old data wiped')
# for i in range(74):
#     filename = f"patent_chunk_{i+1}.csv"
#     session.run(import_patent_query(filename))
#     print("Finished importing "+filename)
# session.run(remove_na_org)
# print('Data Import Complete')

Old data wiped
Finished importing patent_chunk_1.csv
Finished importing patent_chunk_2.csv
Finished importing patent_chunk_3.csv
Finished importing patent_chunk_4.csv
Finished importing patent_chunk_5.csv
Finished importing patent_chunk_6.csv
Finished importing patent_chunk_7.csv
Finished importing patent_chunk_8.csv
Finished importing patent_chunk_9.csv
Finished importing patent_chunk_10.csv
Finished importing patent_chunk_11.csv
Finished importing patent_chunk_12.csv
Finished importing patent_chunk_13.csv
Finished importing patent_chunk_14.csv
Finished importing patent_chunk_15.csv
Finished importing patent_chunk_16.csv
Finished importing patent_chunk_17.csv


## Import and merge SBIR data

In [63]:
session.run(import_sbir_query('sbir_filtered.csv'))

In [32]:
show_result("""MATCH (n) RETURN labels(n) AS Label, count(n) AS Count;""")

[{'Label': ['Inventor'], 'Count': 427630}, {'Label': ['Patent'], 'Count': 192307}, {'Label': ['Organization'], 'Count': 45340}, {'Label': ['SBIR_Award'], 'Count': 97}, {'Label': ['PatentKind'], 'Count': 6}]


In [73]:
org_query = """MATCH (o:Organization)-[:FILED]->(p:Patent) 
RETURN o.name AS Organization, p.doc_number AS PatentNumber 
LIMIT 10;"""

show_result(org_query)

[{'Organization': "'NIKE, Inc.]']", 'PatentNumber': "'D0974016']"}, {'Organization': "'NIKE, Inc.]']", 'PatentNumber': "'11541624']"}, {'Organization': "'HUBLOT SA, Genéve]']", 'PatentNumber': "'D0974187']"}, {'Organization': "'MITSUBISHI HEAVY INDUSTRIES MACHINERY SYSTEMS, LTD.]']", 'PatentNumber': "'11541565']"}, {'Organization': "'Oliver Packaging and Equipment Company]']", 'PatentNumber': "'11541566']"}, {'Organization': "'N/A']", 'PatentNumber': "'11541567']"}, {'Organization': "'N/A']", 'PatentNumber': "'D0974189']"}, {'Organization': "'N/A']", 'PatentNumber': "'11541616']"}, {'Organization': "'N/A']", 'PatentNumber': "'11541631']"}, {'Organization': "'Hewlett-Packard Development Company, L.P.]']", 'PatentNumber': "'11541568']"}]


## Queries used in Neo4j Browser for visualizations

In [ ]:
# Query to show important elements of entire network
"""
MATCH (n1:SBIR_Award) - [r1] - (n2:Inventor) - [r2] - (n3:Patent) - [r3] - (n4:Organization)
RETURN n1,r1,n2,r2,n3,r3,n4
"""
# Query for organization network
"""
MATCH (n1:SBIR_Award) - [r1] - (n2:Inventor) - [r2] - (n3:Patent) - [r3] - (n4:Organization{name:"'GOOGLE LLC'"})
RETURN n1,r1,n2,r2,n3,r3,n4
"""
# Query for individual network
"""
MATCH (n1:SBIR_Award) - [r1] - (n2:Inventor{name: "Michael Jones"}) - [r2] - (n3:Patent) - [r3] - (n4:Organization)
RETURN n1,r1,n2,r2,n3,r3,n4
"""

### Community detection attempt (failure)

https://neo4j.com/docs/graph-data-science/current/algorithms/louvain/

In [ ]:
graph = "communities"
delete_graph_if_exists(graph)
g, project_result = gds.graph.project("communities", ["Inventor", "Patent", "SBIR_Award"], {"INVENTED":{'orientation':'UNDIRECTED'}})

In [ ]:
df = gds.louvain.write(g, writeProperty='louvainCommunityId')

In [ ]:
#For each cluster (community), display the list of people belonging to that cluster
gds.run_cypher(
"""
MATCH (p:Person)
WITH p
RETURN p.louvainCommunityId, collect(p.name)
""")

In [ ]:
find_large_comm_id = """
                    CALL gds.wcc.stream('subgraph')
                    YIELD nodeId, componentId
                    WITH componentId, count(nodeId) AS componentSize
                    ORDER BY componentSize DESC
                    LIMIT 1
                    RETURN componentId;
                    """

In [ ]:
session.run(find_large_comm_id)